In [1]:
import numpy as np
import glob
import scipy.io as sio
import torch
from torch import nn
import csv
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import yaml
from XRF55_Dataset import XRF55_Datase 

### Loda Data

In [2]:
train_dataset = XRF55_Datase(root_dir="D:\Data\XRF55\XRF_dataset", scene='all', is_train=True)
test_dataset = XRF55_Datase(root_dir="D:\Data\XRF55\XRF_dataset", scene='all', is_train=False)

In [3]:
def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.

    dict_keys(['modality', 'scene', 'subject', 'action', 'idx', 'output', 
    'input_rgb', 'input_depth', 'input_lidar', 'input_mmwave'])
    '''
    # ## get sequence lengths
    # for t in batch:
    #     print(t[0],t[1].shape,t[2].shape,t[3])
    #     print(a)
    # # #     # print(t[0].shape,t[1].shape)
    # # kpts = []
    # # [kpts.append(np.array(t['output'])) for t in batch]
    # # kpts = torch.FloatTensor(np.array(kpts))

    # # lengths = torch.tensor([t['input_rgb'].shape[0] for t in batch ])
    # all_actions = {'A01': 0., 'A02': 1., 'A03': 2., 'A04': 3., 'A05': 4., 
    #             'A06': 5., 'A07': 6., 'A08': 7., 'A09': 8., 'A10': 9.,
    #             'A11': 10., 'A12': 11., 'A13': 12., 'A14': 13., 'A15': 14., 
    #             'A16': 15., 'A17': 16., 'A18': 17., 'A19': 18., 'A20': 19., 
    #             'A21': 20., 'A22': 21., 'A23': 22., 'A24': 23., 'A25': 24., 
    #             'A26': 25., 'A27': 26.}
    
    labels = []
    [labels.append(float(t[3])) for t in batch]
    labels = torch.FloatTensor(labels)

    # wifi-csi
    rfid_data = np.array([(t[1]) for t in batch ])
    rfid_data = torch.FloatTensor(rfid_data)

    return rfid_data, labels

In [4]:
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn_padd)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn_padd)

In [5]:
for data in train_dataloader:
    rfid_data, labels = data
    print(rfid_data.shape)
    print(labels)
    break

torch.Size([16, 23, 148])
tensor([20.,  6., 45., 54.,  4., 46., 53., 19., 42., 11., 28., 45., 53., 43.,
        46., 52.])


### Model

In [23]:
import sys
path = os.getcwd()
os.chdir('c:/Users/Chen_Xinyan/Desktop/Modality_Invariant/XRF55')

# from backbone_models.RFID.BiLSTM import *
from backbone_models.RFID.ResNet import *
# from HAR_CSI_benchmark.models.mynetwork import *
# wifi_data = torch.randn(32, 3, 114, 10)
# model = rfid_BiLSTM(num_classes=55)
model = resnet18()

# out = model(wifi_data)
# print(out.shape)

In [24]:
print(model)

ResNet(
  (conv1): Conv1d(23, 128, kernel_size=(7,), stride=(2,), padding=(3,), bias=False)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2):

### training 

In [25]:
def test(model, tensor_loader, criterion, device):
    model.eval()
    test_acc = 0
    test_loss = 0
    for data in tqdm(tensor_loader):
        wifi_data, labels = data
        inputs = wifi_data.to(device)
        labels.to(device)
        labels = labels.type(torch.LongTensor)
        outputs = model(inputs)
        outputs = outputs.type(torch.FloatTensor)
        outputs.to(device)
        loss = criterion(outputs,labels)
        predict_y = torch.argmax(outputs,dim=1).to(device)
        accuracy = (predict_y == labels.to(device)).sum().item()
        test_acc += accuracy
        test_loss += loss.item() * labels.size(0)
        outputs = outputs.detach().numpy()
        labels = labels.detach().numpy()
    test_acc = test_acc/len(tensor_loader.dataset)
    test_loss = test_loss/len(tensor_loader.dataset)
    print("validation accuracy:{:.4f}, loss:{:.5f}".format(float(test_acc),float(test_loss)))
    return test_acc

In [26]:
def train(model, train_loader, test_loader, num_epochs, learning_rate, criterion, device):
    optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)
    # optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[20,40],gamma=0.1)
    parameter_dir = './backbone_models/RFID/ResNet18.pt'
    best_test_acc = 0
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        epoch_accuracy = 0.0
        # num_iter = 2000
        for i, data in enumerate(tqdm(train_loader)):
            # if i < num_iter:
            rgb_data, labels = data
            inputs = rgb_data.to(device)
            labels.to(device)
            labels = labels.type(torch.LongTensor)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            outputs = outputs.type(torch.FloatTensor)
            outputs.to(device)
            # print(outputs)
            # print(labels)
            loss = criterion(outputs,labels)
            # print(loss)
            # if loss == float('nan'):
            #     print('nan')
            #     print(outputs)
            #     print(labels)
            loss.backward()
            # print(length)
            # print("loss is ", loss.item())
            optimizer.step()

            
            epoch_loss += loss.item() * labels.size(0)
            predict_y = torch.argmax(outputs,dim=1).to(device)
            epoch_accuracy += (predict_y == labels.to(device)).sum().item()
            outputs = outputs.detach().numpy()
            labels = labels.detach().numpy()
            # else:
                # break
            # print("epoch loss is ", epoch_loss)
        # epoch_loss = epoch_loss/(rgb_data.size(0)*len(train_loader.dataset))
        # epoch_accuracy = epoch_accuracy/(rgb_data.size(0)*len(train_loader.dataset))
        epoch_loss = epoch_loss/len(train_loader.dataset)
        epoch_accuracy = epoch_accuracy/len(train_loader.dataset)
        print('Epoch:{}, Accuracy:{:.4f},Loss:{:.9f}'.format(epoch+1, float(epoch_accuracy),float(epoch_loss)))
        if (epoch+1) % 10 == 0:
            test_acc = test(
                model=model,
                tensor_loader=test_loader,
                criterion = criterion,
                device= device
            )
            if test_acc >= best_test_acc:
                print(f"best test acuracy is:{test_acc}")
                best_test_acc = test_acc
                torch.save(model, parameter_dir)
        scheduler.step()
    return

In [27]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device) 

cuda:0


In [28]:
criteria = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.load_state_dict(torch.load('./HAR_lidar_benchmark/lidar_all_random.pt'))
model.to(device)
train(
    model=model, 
    train_loader= train_dataloader,
    test_loader= test_dataloader,    
    num_epochs= 100,
    learning_rate=1e-3,
    criterion = criteria,
    device=device 
    )

  0%|          | 0/963 [00:00<?, ?it/s]

100%|██████████| 963/963 [57:22<00:00,  3.58s/it]


Epoch:1, Accuracy:0.0229,Loss:4.062083960


100%|██████████| 963/963 [55:07<00:00,  3.43s/it]


Epoch:2, Accuracy:0.0263,Loss:3.949326643


...

Epoch:50, Accuracy:0.9779,Loss:0.113987292


100%|██████████| 207/207 [15:27<00:00,  4.48s/it]


validation accuracy:0.4380, loss:2.72519


 43%|████▎     | 416/963 [13:11<17:21,  1.90s/it]


KeyboardInterrupt: 

In [8]:
criteria = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load('./HAR_lidar_benchmark/lidar_all_random.pt'))
model.to(device)
test(
        model=model,
        tensor_loader=val_loader,
        criterion = criteria,
        device= device
    )

100%|██████████| 4010/4010 [10:04<00:00,  6.63it/s]

validation accuracy:0.7845, loss:0.75000


0.7845429604688864

In [11]:
parameter_dir = './HAR_lidar_benchmark/lidar_all_random.pt'
torch.save(model.state_dict(), parameter_dir)